# Employee Scheduling System

In [48]:
import gdown
import pandas as pd
from pulp import *

url = "https://drive.google.com/uc?id=15BPH7-3GGWBfXPJQ3stkT6SHQECbT-pt"
output = "shifts.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv("shifts.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?id=15BPH7-3GGWBfXPJQ3stkT6SHQECbT-pt
To: C:\Users\Lenovo\shifts.csv
100%|█████████████████████████████████████████████████| 298/298 [00:00<?, ?B/s]


In [49]:
df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

a = df.drop(index=["Wage rate per 9h shift ($)"], columns=["Workers Required"]).values
a

array([[1, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 1]], dtype=object)

In [50]:
# number of shifts
n = a.shape[1]

# number of time windows
T = a.shape[0]

# number of workers required per time window
d = df["Workers Required"].values

# wage rate per shift
w = df.loc["Wage rate per 9h shift ($)", :].values.astype(int)

In [51]:
# Decision variables
y = LpVariable.dicts("num_workers", list(range(n)), lowBound=0, cat="Integer")

In [52]:
#Minimize the total cost of wages paid to all workers
# Create problem
prob = LpProblem("scheduling_workers", LpMinimize)

In [53]:
prob += lpSum([w[j] * y[j] for j in range(n)])
for t in range(T):
    prob += lpSum([a[t, j] * y[j] for j in range(n)]) >= d[t]

In [54]:
#Solve
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Optimal


In [55]:
for shift in range(n):
    print(
        f"The number of workers needed for shift {shift} is {int(y[shift].value())} workers"
    )

The number of workers needed for shift 0 is 46 workers
The number of workers needed for shift 1 is 23 workers
The number of workers needed for shift 2 is 38 workers
The number of workers needed for shift 3 is 30 workers
